# Reading TXT File

In [2]:
# Text based data Ingestion
from langchain_community.document_loaders import TextLoader
loader = TextLoader("data/anees_dataset.txt")
text_docs = loader.load()

# Scraping Anees School Website

In [3]:
# Web Based Data Ingestion
from langchain_community.document_loaders import WebBaseLoader

# Load, chunk, and Index the contents of a Webpage
loader = WebBaseLoader(
    ["https://aneesschool.com/life-at-anees/", "https://aneesschool.com/aims-objectives/", "https://aneesschool.com/vision-mission/", "https://aneesschool.com/academic-building/", "https://aneesschool.com/anees-core-team/", "https://aneesschool.com/message-from-ceo-cum-chairman/", "https://aneesschool.com/the-formative-years/", "https://aneesschool.com/the-junior-school/", "https://aneesschool.com/the-middle-school/", "https://aneesschool.com/the-senior-school/", "https://aneesschool.com/personality-development/", "https://aneesschool.com/creative-skills/", "https://aneesschool.com/important-tips/", "https://aneesschool.com/student-participation/", "https://aneesschool.com/digital-campus/", "https://aneesschool.com/health-hygiene/", "https://aneesschool.com/transport-safety/", "https://aneesschool.com/auditorium/", "https://aneesschool.com/computer-lab/", "https://aneesschool.com/library/", "https://aneesschool.com/art-room/", "https://aneesschool.com/science-laboratories/", "https://aneesschool.com/sports/", "https://aneesschool.com/smart-spacious-classes/"]
)
web_documents = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Loading PDF and its related data

In [4]:
# PDF Data Ingestion
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/Prospectus.pdf")
docs = loader.load()

# Chuncking all the data to meet the context length of the LLM Models

In [5]:
# Converting Text to chuncks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(text_docs)

In [6]:
# Converting PDF text to chuncks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)

In [7]:
# Converting Web text to chuncks
from langchain_text_splitters import RecursiveCharacterTextSplitter
web_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
web = web_splitter.split_documents(web_documents)

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
dataBase1 = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))
dataBase2 = FAISS.from_documents(web, OllamaEmbeddings(model="nomic-embed-text"))
dataBase3 = FAISS.from_documents(text, OllamaEmbeddings(model="nomic-embed-text"))

In [9]:
query = "Fees"
result = dataBase3.similarity_search(query)
result[0].page_content

'Here is our current fee structure: Nursery - 4200, LKG - 4200, UKG - 4250, First - 4400, Second - 4450, Third - 4500, Fourth - 4550, Fifth - 4600, Sixth - 4700, Seventh - 4800, Eighth - 4900, Ninth - 5000, Tenth - 5100, +1 & +2 Arts/Commerce - 5000, +1 & +2 Non-Med/Med - 5750. Please note that these fees are subject to change with prior notice. Additional charges may apply for examinations, extracurricular activities, and optional facilities.\n\nOur fee structure is as follows: Nursery - 4200, LKG - 4200, UKG - 4250, First - 4400, Second - 4450, Third - 4500, Fourth - 4550, Fifth - 4600, Sixth - 4700, Seventh - 4800, Eighth - 4900, Ninth - 5000, Tenth - 5100, +1 & +2 Arts/Commerce - 5000, +1 & +2 Non-Med/Med - 5750. Please note that fees are subject to revision, and additional charges may apply for examinations, extracurricular activities, and optional facilities.'

In [ ]:
# Importing LLM and Implementing
from langchain_ollama import ChatOllama
# Load Ollama LLaMa 3
ollama = ChatOllama(model="llama3")
ollama

ChatOllama(model='llama3.2')

In [19]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question asked by the user only in the provided context.
    Think step by step before answering questions and provide a detailed answer.
    I will tip you $1000 if the user finds the answer helpful
    
    Character: AneesBot, a formal and polite AI assistant employed by Anees School.
    
    Task:
    Respond to user inquiries about Anees School and its facilities in a professional, informative, and helpful manner. 
    Provide detailed and accurate information, adhering to the provided context. 
    Maintain a polite and courteous tone throughout the interaction, addressing all interactions with prospective parents beginning with "Dear Parent," followed by the appropriate information.
    Aim to keep responses within **500 characters**. If necessary to provide a comprehensive and informative response, the character limit may be exceeded, but only when absolutely required and by a reasonable margin.
    
    Guidelines:
    Professionalism: Maintain a professional and courteous demeanor, avoiding slang or informal language. 
    Accuracy: Ensure the accuracy and relevance of all information provided.
    Conciseness: Deliver information in a clear, concise, and easy-to-understand manner.
    Context Awareness: If a user query falls outside the scope of school-related information, politely remind them of the context and request a rephrased question. 
    For example: "Dear Parents, to best assist you, please limit your inquiries to questions about Anees School and its facilities. Could you please rephrase your question?"

    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [12]:
# Create stuff document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
documentChain = create_stuff_documents_chain(ollama, prompt)

In [13]:
retriever1 = dataBase1.as_retriever()
retriever2 = dataBase2.as_retriever()
retriever3 = dataBase3.as_retriever()

In [14]:
from langchain.retrievers import MergerRetriever
merger_retriever = MergerRetriever(retrievers=[retriever1, retriever2, retriever3])
merger_retriever

MergerRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000273310ADE50>, search_kwargs={}), VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000273310AE7E0>, search_kwargs={}), VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000273310AEDB0>, search_kwargs={})])

In [15]:
# Retriever Chain
from langchain.chains import create_retrieval_chain
retrievalChain = create_retrieval_chain(merger_retriever, documentChain)

In [24]:
retrievalChain.invoke({"input":"Why should I trust you?"})

{'input': 'Why should I trust you?',
 'context': [Document(id='269b1df3-7543-4bef-b23e-c5d9f71b60c6', metadata={'source': 'data/Prospectus.pdf', 'page': 12}, page_content="Education is the process of instruction aimed at the all round   development. It \ndispels ignorance. At Anee's, learning includes imbibing moral values, steeling of \ncharacter and building mental strength.\nOur educational programme fosters team spirit and promotes co-operation. These \nvalues are considered sacred and precious.\nWe are conﬁdent that apart from having knowledge and excelling in academics, our \nstudents become individuals with integrity and sense of purpose.\nThus, we at Anee's strive to provide our students:\nAn insight to the world and their surroundings\uf09f\nKnowledge about their responsibilities towards the society and mother earth\uf09f\nImportance of all the National and International special days\uf09f\nOrganise public campaigns to educate the society about national and \uf09f\ninternation